# Modelación de datos  - Proyecto Integrador

Este notebook presenta como se realizo la modelación y el entrenamiento de varios de modelos de clasificación de sentimientos. Esta modelación se realiza a partir de la tabla analitica obtenida en el notebook de data preparation en el que a partir del analisis de mineria de texto se llega al data frame que contiene todos los tweets (filas) y las palabras que aparecen en el bag of words (columnas) a partir de la limpieza, tokenización, remoción de stop words, lemma. Al final como se presento en el anterior notebook esta matriz utiliza el calculo del tf-idf para su construcción.

Los estudiantes que participaron en el desarrollo de este proyecto son:

1. Jorge Luis Renteria
2. Edgar Leandro Jimenez
3. Jesus Alberto Arcia

Universidad Eafit, 2019.



### Importacion de librerias necesarias¶

In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import glob

### Carga de datos

In [7]:
df=pd.read_csv("bow_50.csv")
df=df.drop(columns='Unnamed: 0',axis=1)
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,3056,3057,3058,3059,3060,3061,3062,3063,3064,label
0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.130454,0.130454,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.000000,0.098873,0.098873,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.0,0.605844,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.000000,0.000000,0.228969,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


### Estructuracion data entrenamiento

In [8]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]
y=['5' if x >10 else x for x in y]
y=['a' if float(x) <5 else 'b' for x in y]
xEntrenamiento, xPrueba, yEntrenamiento, yPrueba = train_test_split(X,y, test_size=0.2)

## ENTRENAMIENTO DEL MODELO

### Cargamos los modelos de clasificación a utilizar

In [10]:
RFcla=RandomForestClassifier()
SVMcla=SVC()
GBcla=GradientBoostingClassifier()
DTcla=DecisionTreeClassifier()
knn=KNeighborsClassifier()

### Se realiza el entrenamiento de cada uno de los modelos

In [11]:
RFcla.fit(xEntrenamiento,yEntrenamiento)
SVMcla.fit(xEntrenamiento,yEntrenamiento)
GBcla.fit(xEntrenamiento,yEntrenamiento)
DTcla.fit(xEntrenamiento,yEntrenamiento)
knn.fit(xEntrenamiento,yEntrenamiento)

type(RFcla.predict(xPrueba))
type(yPrueba)

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda2\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


list

In [ ]:
### Entrenamos un Knn con diferentes vecinos

In [12]:
for k in list(range(1,4)):
    knn=KNeighborsClassifier(n_neighbors=k)
    knn.fit(xEntrenamiento,yEntrenamiento)
    precision=0

    for i,val in enumerate(knn.predict(xPrueba)):
        if val == pd.DataFrame(yPrueba)[0].iloc[i]:
            precision=precision+1
    print('Vecinos')
    print((precision/len(yPrueba)))

Vecinos
0.5420454545454545
Vecinos
0.5693181818181818
Vecinos
0.5454545454545454


### Calculamos precisión para los modelos entrenados

In [13]:
precision=0
for i,val in enumerate(RFcla.predict(xPrueba)):
    if val == pd.DataFrame(yPrueba)[0].iloc[i]:
        precision=precision+1
print('Random forest')
print((precision/len(yPrueba)))

Random forest
0.6022727272727273


In [14]:
precision=0
for i,val in enumerate(SVMcla.predict(xPrueba)):
    if val == pd.DataFrame(yPrueba)[0].iloc[i]:
        precision=precision+1
print('SVM')
print((precision/len(yPrueba)))

SVM
0.6056818181818182


In [15]:
precision=0
for i,val in enumerate(GBcla.predict(xPrueba)):
    if val == pd.DataFrame(yPrueba)[0].iloc[i]:
        precision=precision+1
print('Gradiente Boosting')
print((precision/len(yPrueba)))

Gradiente Boosting
0.6056818181818182


In [16]:
precision=0
for i,val in enumerate(DTcla.predict(xPrueba)):
    if val == pd.DataFrame(yPrueba)[0].iloc[i]:
        precision=precision+1
print('DecisoinTree')
print((precision/len(yPrueba)))

DecisoinTree
0.5511363636363636
